In [1]:
from __future__ import division
import os
import cv2
import numpy as np
import sys
import pickle
from optparse import OptionParser
import time
import tensorflow as tf
from keras_frcnn import config
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers
import keras_frcnn.vgg as nn

In [2]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
sys.setrecursionlimit(40000)

In [3]:
config_filename="./config_file/config.pickle"

with open(config_filename, 'rb') as f_in:
    C = pickle.load(f_in)

class_mapping = C.class_mapping
C.model_path = './config_file/model_frcnn.hdf5'
if 'bg' not in class_mapping:
    class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
print(class_mapping)
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}

{0: 'chair', 1: 'dog', 2: 'pottedplant', 3: 'bottle', 4: 'bird', 5: 'sofa', 6: 'person', 7: 'cat', 8: 'bicycle', 9: 'boat', 10: 'tvmonitor', 11: 'car', 12: 'train', 13: 'aeroplane', 14: 'horse', 15: 'diningtable', 16: 'cow', 17: 'bus', 18: 'motorbike', 19: 'sheep', 20: 'bg'}


In [4]:
from moviepy.editor import VideoFileClip

video_path = './dataset/youtube.mp4'
clip = VideoFileClip(video_path)
new_video = video_path.split("/")
new_video[-1] = "detected_"+new_video[-1]
new_video = "/".join(new_video)

In [5]:
def format_img_size(img, C):
    """ formats the image size based on config """
    img_min_side = float(C.im_size)
    (height, width ,_) = img.shape

    if width <= height:
        ratio = img_min_side/width
        new_height = int(ratio * height)
        new_width = int(img_min_side)
    else:
        ratio = img_min_side/height
        new_width = int(ratio * width)
        new_height = int(img_min_side)
    img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    return img, ratio

def format_img_channels(img, C):
    """ formats the image channels based on config """
    img = img[:, :, (2, 1, 0)]
    img = img.astype(np.float32)
    img[:, :, 0] -= C.img_channel_mean[0]
    img[:, :, 1] -= C.img_channel_mean[1]
    img[:, :, 2] -= C.img_channel_mean[2]
    img /= C.img_scaling_factor
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, axis=0)
    return img

def format_img(img, C):
    """ formats an image for model prediction based on config """
    img, ratio = format_img_size(img, C)
    img = format_img_channels(img, C)
    return img, ratio

# Method to transform the coordinates of the bounding box to its original size
def get_real_coordinates(ratio, x1, y1, x2, y2):

    real_x1 = int(round(x1 // ratio))
    real_y1 = int(round(y1 // ratio))
    real_x2 = int(round(x2 // ratio))
    real_y2 = int(round(y2 // ratio))

    return (real_x1, real_y1, real_x2 ,real_y2)

In [6]:
img_input = Input(shape=(None, None, 3))
roi_input = Input(shape=(C.num_rois, 4))
feature_map_input = Input(shape=(None, None, 512))
shared_layers = nn.nn_base(img_input, trainable=False)

In [7]:
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn_layers = nn.rpn(shared_layers, num_anchors)

model_rpn = Model(img_input, rpn_layers)
classifier = nn.classifier(feature_map_input, roi_input, C.num_rois, nb_classes=len(class_mapping), trainable=True)
model_classifier_only = Model([feature_map_input, roi_input], classifier)

print('Loading weights from {}'.format(C.model_path))
result1=model_rpn.load_weights(C.model_path, by_name=True)
result2 = model_classifier_only.load_weights(C.model_path, by_name=True)

Loading weights from ./config_file/model_frcnn.hdf5


In [8]:
all_imgs = []

bbox_threshold = 0.8

visualise = True

for img in clip.iter_frames():

    st = time.time()
    X, ratio = format_img(img, C)

    if K.image_data_format() == 'channels_last':
        X = np.transpose(X, (0, 2, 3, 1))

    # get the feature maps and output from the RPN
    [Y1, Y2, F] = model_rpn.predict(X)


    R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_data_format(), overlap_thresh=0.7)

    # convert from (x1,y1,x2,y2) to (x,y,w,h)
    R[:, 2] -= R[:, 0]
    R[:, 3] -= R[:, 1]

    # apply the spatial pyramid pooling to the proposed regions
    bboxes = {}
    probs = {}

    for jk in range(R.shape[0]//C.num_rois + 1):
        ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
        if ROIs.shape[1] == 0:
            break

        if jk == R.shape[0]//C.num_rois:
            #pad R
            curr_shape = ROIs.shape
            target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
            ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
            ROIs_padded[:, :curr_shape[1], :] = ROIs
            ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
            ROIs = ROIs_padded

        [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

        for ii in range(P_cls.shape[1]):

            if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
                continue

            cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

            if cls_name not in bboxes:
                bboxes[cls_name] = []
                probs[cls_name] = []

            (x, y, w, h) = ROIs[0, ii, :]

            cls_num = np.argmax(P_cls[0, ii, :])
            try:
                (tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
                tx /= C.classifier_regr_std[0]
                ty /= C.classifier_regr_std[1]
                tw /= C.classifier_regr_std[2]
                th /= C.classifier_regr_std[3]
                x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
            except:
                pass
            bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
            probs[cls_name].append(np.max(P_cls[0, ii, :]))

    all_dets = []

    for key in bboxes:
        bbox = np.array(bboxes[key])

        new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.5)
        for jk in range(new_boxes.shape[0]):
            (x1, y1, x2, y2) = new_boxes[jk,:]

            (real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)

            cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),2)

            textLabel = '{}: {}'.format(key,int(100*new_probs[jk]))
            all_dets.append((key,100*new_probs[jk]))

            (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
            textOrg = (real_x1, real_y1-0)

            cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
            cv2.rectangle(img, (textOrg[0] - 5,textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
            cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)
    

    print('Elapsed time = {}'.format(time.time() - st))
    print(all_dets)
    all_imgs.append(img)
    

Elapsed time = 3.899998664855957
[('person', 98.71805310249329)]
Elapsed time = 0.5602397918701172
[('person', 97.93395400047302)]
Elapsed time = 0.546663761138916
[('person', 98.45598936080933)]
Elapsed time = 0.5601019859313965
[('person', 98.65182042121887)]
Elapsed time = 0.5594336986541748
[('person', 98.63329529762268)]
Elapsed time = 0.6689362525939941
[('person', 98.59493374824524)]
Elapsed time = 0.5396571159362793
[('person', 98.31007719039917)]
Elapsed time = 0.5389735698699951
[('person', 97.63058423995972), ('person', 85.13267636299133)]
Elapsed time = 0.5439131259918213
[('person', 97.34805822372437)]
Elapsed time = 0.5460467338562012
[('person', 99.16512370109558)]
Elapsed time = 0.5457901954650879
[('person', 99.13397431373596)]
Elapsed time = 0.5378715991973877
[('person', 98.7636923789978)]
Elapsed time = 0.5396392345428467
[('person', 96.90778255462646)]
Elapsed time = 0.5582592487335205
[('person', 94.9280321598053)]
Elapsed time = 0.55680251121521
[('person', 97.55

In [9]:
from moviepy.editor import ImageSequenceClip
new_clips = ImageSequenceClip(all_imgs, fps=24)
new_clips.write_videofile(new_video, fps=24)

t:   0%|          | 0/4440 [00:00<?, ?it/s, now=None]

Moviepy - Building video ./dataset/detected_youtube.mp4.
Moviepy - Writing video ./dataset/detected_youtube.mp4



Moviepy - Done !
Moviepy - video ready ./dataset/detected_youtube.mp4
